In [2]:
%reload_ext autoreload
%autoreload 2
os.chdir("..")
import toml
from core.dl_framework.utils import read_config
%%bash 
pwd

In [11]:
config_file = toml.load("configs/default_train_config.toml")
setup_config = read_config(config_file)

In [115]:
from core.dl_framework.learner import Learner
from core.dl_framework.data import get_dataset, Dataset, DataLoader, split_data
bs = setup_config["h_batch_size"]
x_train, y_train, x_test, y_test = get_dataset(setup_config["s_source"], setup_config["s_set"], setup_config["p_tmp_data_path"])
train_ds, test_ds =  Dataset(x_train, y_train), Dataset(x_test, y_test)
train_dl, valid_dl = DataLoader(train_ds, bs), DataLoader(test_ds, bs)

In [135]:
learn = Learner(train_ds, setup_config)
vars(learn)
# import matplotlib.pyplot as plt
# plt.imshow(learn.data.valid_ds[1][0].float().reshape(28, -1))
#train_ds, valid_s = split_trainset_ds(train_ds, 0.2)
#len(train_ds)
# len(train_ds)
# len(learn.data[1])




{'model': 'Model_2',
 'opt': 'adam',
 'loss': 'cross_entropy',
 'bs': 64,
 'data': <core.dl_framework.data.DataBunch at 0x7fd88c45e400>,
 'cbh': <core.dl_framework.callbacks.CallbackHandler at 0x7fd88c45e320>,
 'recorder': core.dl_framework.callbacks.Recorder_Cb,
 '_setup_config': {'m_gpu': False,
  'p_tmp_data_path': './tmp_files',
  'g_num_epochs': 100,
  'g_num_classes': 10,
  'g_valid_split': 0.2,
  'g_loss_func': 'cross_entropy',
  'g_optimizer': 'adam',
  'g_arch': 'Model_2',
  's_source': 'torchvision',
  's_set': 'MNIST',
  'h_batch_size': 64,
  'h_lr': 1e-05,
  'c_m_monitor': ['valid_acc', 'valid_loss', 'loss'],
  'c_m_print2console': True,
  'c_e_monitor': 'valid_loss',
  'c_e_patience': 1000}}

In [110]:
train_dl.dataset

In [79]:
class test_class():
    def __init__(self, a, b):
        self.a = a
        self.b = self.func(b)

    def func(self):
        return arg+1
vars(test_class(1, 2))

TypeError: func() takes 1 positional argument but 2 were given